#### Faiss - created by Facebook
- https://docs.langchain.com/oss/python/integrations/vectorstores#faiss

In [ ]:
# perform similarity search using ollamaEmbeddings and FAISS vector store db

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader = TextLoader("speech.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(texts, embeddings)

query = "What is the main topic of the speech?"

results = db.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.



#### Converting vector store to a Retriever Class
- https://docs.langchain.com/oss/python/integrations/retrievers

In [13]:
retriever = db.as_retriever()
results = retriever.invoke(query)

for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.



#### Similarity search with score
Returned score:
- lower = better (distance)
- 0.0 ≈ perfect match

In [16]:
score = db.similarity_search_with_score(query)
score

[(Document(id='1a05ca02-bcf2-4148-af90-f7038bba1782', metadata={'source': 'speech.txt'}, page_content='Freedom was not gifted; it was earned through courage and sacrifice.\nCountless voices rose together to demand dignity and self-rule.\nEvery step toward independence carried the weight of hope and loss.\nThe struggle taught us unity beyond language, region, or belief.\nIndependence is not just a date, but a responsibility we carry daily.\nIt reminds us to protect justice, equality, and truth.\nThe past whispers lessons of resilience and bravery.'),
  np.float32(0.8328067)),
 (Document(id='c7f6572c-a69c-4b27-b7f5-a50a4ff13ec5', metadata={'source': 'speech.txt'}, page_content='The present asks us to build with integrity and compassion.\nThe future depends on how wisely we use our freedom today.\nIndependence lives on when we choose progress over fear.'),
  np.float32(0.87817836))]

#### converting the query to a vector and perform similarity search with vectors instead of query sentences

In [21]:
vector = embeddings.embed_query(query) 

result = db.similarity_search_by_vector(vector)

for i, res in enumerate(result):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.



#### saving the vector store db in local

In [22]:
# saving and loading
db.save_local("faiss_index")

In [26]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
result = new_db.similarity_search(query)

for i, res in enumerate(result): 
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

